<a href="https://colab.research.google.com/github/bahkali/A4Products/blob/master/Dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-end Multi-class Dog Bread Classification

This notebook build an end-to-end multi-class images classifier using Tensorflow 2.0 and TensorFlow Hub.

## 1. Problem

Identifying the bread of dog given an image of a dog.
when I'm sitting at the cafe and I take a phote of a dog, I want to know what bread of dog it is.

## 2. Data

The data we're using is from Kaggle's dog breed identification competition.


## 3. Evaluation

The evaluation is a file with prediction probabilities for each dog breed each test image.

## 4. Features

Some information about the data:
* We're dealing with images (unstructured data) so it's probably best we use deep learning/transfer learning. 
* There are 120 breeds of dogs (this means there are 120 different classes).
* There are around 10000 images in the training set (Images have labels)
* There are around 10000 images in the test set (Images have no labels, because we'll want to predict them).

In [2]:
# Import Tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Import Tensorflow into colab
import tensorflow as tf
import tensorflow_hub as hub


In [3]:
print("TF version:", tf.__version__)

TF version: 2.4.0


In [4]:
print("TF Hub version:", hub.__version__)

TF Hub version: 0.11.0


In [5]:
# Check for GPU 
print("GPU", "available (YESSS!!!!" if tf.config.list_physical_devices("GPU") else "not available :(")

GPU available (YESSS!!!!
